In [1]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
print("Python: {0}".format(sys.version))


################################################################################
## General Stuff
################################################################################
from ioUtils import saveJoblib, loadJoblib, saveFile, getFile
import urllib
from urllib.parse import quote
from timeUtils import clock, elapsed
from fsUtils import moveFile, setFile, setDir, setSubDir, isFile, isDir, mkDir
from fileUtils import getFileBasics, getBasename
from webUtils import getHTML
from time import sleep
import time
from multiprocessing import Pool


######################################################
# Music Stuff
######################################################
from dbBase import dbBase
from multiArtist import multiartist
from myMusicDBMap import myMusicDBMap
from mainDB import mainDB
from parseDBArtistsData import parseDBArtistsData




import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

Python: 3.7.7 (default, Mar 26 2020, 10:32:53) 
[Clang 4.0.1 (tags/RELEASE_401/final)]


/Users/tgadfort/opt/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


Notebook Last Run Initiated: 2020-10-13 21:41:15.472503


## Parse Artists Base

In [2]:
%load_ext autoreload
%autoreload
from mainDB import mainDB
mdb    = myMusicDBMap(debug=True)
maindb = mainDB(mdb=mdb, create=True, debug=False)
pdb    = parseDBArtistsData(maindb=maindb)
dbdata = maindb.dbdata

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Creating myMusicDBMap()
   Loading my music db map: /Users/tgadfort/opt/anaconda3/envs/py37/musicdb/myMusicMap.p
   DB keys: ['Discogs', 'AllMusic', 'MusicBrainz', 'AceBootlegs', 'RateYourMusic', 'LastFM', 'DatPiff', 'RockCorner', 'CDandLP', 'MusicStack', 'MetalStorm']
Found 4906 artist entries
Counter({'Discogs': 4758, 'AllMusic': 4615, 'MusicBrainz': 4369, 'LastFM': 3445, 'CDandLP': 764, 'DatPiff': 555, 'RockCorner': 554, 'AceBootlegs': 181, 'RateYourMusic': 68, 'MusicStack': 5, 'MetalStorm': 0})
Counter({'Discogs': 4758, 'AllMusic': 4615, 'MusicBrainz': 4369, 'LastFM': 3445, 'CDandLP': 764, 'DatPiff': 555, 'RockCorner': 554, 'AceBootlegs': 181, 'RateYourMusic': 68, 'MusicStack': 5, 'MetalStorm': 0})
=========================   discogs   =========================
Directory Information:
  Raw:  /Volumes/Piggy/Discog
  Meta: /Users/tgadfort/Music/Discog
  DB:   /Users/tgadfort/Music/Discog
=========

## Create Metadata (Only)

In [ ]:
dbs    = ['AllMusic', 'Discogs', 'MusicBrainz', 'LastFM']
dbs    = []
start,cmt=clock("Parsing Metadata")
for db in dbs:
    pdb.parseMetadata(db=db, nProcs=3, modVals=list(range(100)), force=False) 
elapsed(start, cmt)

## Parse Everything

In [ ]:
dbs    = ['AllMusic', 'Discogs', 'MusicBrainz', 'LastFM']
#dbs = ['MusicBrainz', 'LastFM']
#dbs = ["AllMusic"]
#dbs = ["Discogs", "MusicBrainz"]
dbs = ["LastFM"]
#dbs = []
start,cmt=clock("Parsing Everything")
for db in dbs:
    pdb.parse(db, nProcs=3, modVals=list(range(100)), force=True)
    #pdb.parse(db, nProcs=3, modVals=list(range(78,90)), force=False)
    #pdb.parse(db, nProcs=3, modVals=[0,1,2], force=True)    
elapsed(start, cmt)

## Parse Single ModVal

In [4]:
start,cmt=clock("Parsing Single ModVal")
pdb.parse(db="LastFM", modVals=[0], nProcs=1, force=True)
elapsed(start, cmt)

Current Time is Tue Oct 13, 2020 21:43:57 for Parsing Single ModVal
Parsing LastFM with 1 processes using [[0]] mod values.
Parsing Artist Files For ModVal 0
Forcing Reloads of ModVal=0
  Processing 1125 files.
	Processed 0/1125 files.
	Processed 500/1125 files.
	Processed 1000/1125 files.
Saving 1125 new artist IDs to /Users/tgadfort/Music/Discog/artists-lastfm-db/0-DB.p
  --> This file is 604.7kB.
Saving 1125 new artist IDs name data to /Users/tgadfort/Music/Discog/artists-lastfm-db/metadata/0-Metadata.p
  --> This file is 73.1kB.
Saving 1125 new artist IDs media data to /Users/tgadfort/Music/Discog/artists-lastfm-db/metadata/0-MediaMetadata.p
  --> This file is 355.3kB.
Parsing Artist Extra Files For ModVal 0
  Found 429 extra files for ModVal 0
  Loaded 1125 artist IDs.
Saving 7166 new artist media to /Users/tgadfort/Music/Discog/artists-lastfm-db/0-DB.p
  --> This file is 975.3kB.
Saving 1125 new artist IDs name data to /Users/tgadfort/Music/Discog/artists-lastfm-db/metadata/0-Met

In [ ]:
data = getFile('/Volumes/Piggy/Discog/artists-lastfm-db/0-DB.p')

In [ ]:
data['11792020000']

In [ ]:
dbdata["LastFM"]["Artists"].parseArtistModValExtraFiles(0, force=True, debug=True)

In [ ]:
from glob import glob
glob('/Volumes/Piggy/Discog/artists-lastfm/0/extra/11792020000-*.p')

In [ ]:
bsdata = getHTML(ifile)
ols = bsdata.findAll("ol", {"class": "buffer-standard"})
ols

In [ ]:
ifile='/Volumes/Piggy/Discog/artists-lastfm/0/extra/72742897500-2.p'
#getFile(ifile)
a       = dbdata["LastFM"]["Artist"].getData(ifile, debug=False)

def info(a):
    name    = a.artist.name
    ID      = a.ID.ID
    url     = a.url.url
    pages   = a.pages
    media   = a.media
    counts  = a.mediaCounts
    profile = a.profile

    print("{0: <10}{1}".format("Name:", name))
    print("{0: <10}{1}".format("ID:", ID))
    print("{0: <10}{1}".format("URL:", url))
    print("{0: <10}{1}".format("Pages:", pages.get()))
    print("{0: <10}{1}".format("Profile:", profile.get()))
    print("{0: <10}{1}".format("Counts:", counts.get()))
    print("{0: <10}".format("Media:"))
    for mediaType,mediaData in media.get()['media'].items():
        for mediaInfo in mediaData:
            print("\t{0: <10}{1}".format(mediaType, mediaInfo.get()['album']))

In [ ]:
info(a)

In [ ]:
nProcs = 8
%load_ext autoreload
%autoreload
from mainDB import mainDB
maindb = mainDB(mdb=None, create=True, debug=True)
start,cmt=clock("Parsing All Artists")
start2,cmt2=clock("Parallel Artist Parsing")

parseArtistsParallel(db="AllMusic", nProcs=nProcs)
maindb.setDBFull("AllMusic")

parseArtistsParallel(db="MusicBrainz", nProcs=nProcs)
maindb.setDBFull("MusicBrainz")

parseArtistsParallel(db="Discogs", nProcs=nProcs)
maindb.setDBFull("Discogs")

#parseArtistsParallel(db="AceBootlegs", nProcs=1)
#parseArtistsParallel(db="DatPiff", nProcs=1)
#parseArtistsParallel(db="RateYourMusic", nProcs=3)
parseArtistsParallel(db="LastFM", nProcs=nProcs)
maindb.setDBFull("LastFM")

#parseArtistsParallel(db="RockCorner", nProcs=3)
#parseArtistsParallel(db="CDandLP", nProcs=3)
#parseArtistsParallel(db="MusicStack", nProcs=1)
elapsed(start2, cmt2)
elapsed(start, cmt)

# Master DB

In [ ]:

print("\n\n{0}\n".format("Full DB"))
print("\n\n{0}\n".format("Known DB"))
#maindb.setDBKnown() ## Do this to recreate everything
#maindb.setDBKnown()
#artistDBs = maindb.getKnownArtistDBs()

# Download My Known Artists

In [ ]:
mdb = myMusicDBMap(debug=True)

In [ ]:
dbsToGet = ["AllMusic", "Discogs", "MusicBrainz", "LastFM", "RockCorner", "CDandLP"]
searches = [dbdata["AllMusic"]["Artists"].searchAllMusicForArtist, dbdata["Discogs"]["Artists"].searchDiscogForArtist,
            dbdata["MusicBrainz"]["Artists"].searchMusicBrainzForArtist, dbdata["LastFM"]["Artists"].searchLastFMForArtist,
            dbdata["RockCorner"]["Artists"].searchRockCornerForArtist, dbdata["CDandLP"]["Artists"].searchCDandLPForArtist]

dbsToGet = ["AllMusic"]
searches = [dbdata["AllMusic"]["Artists"].searchAllMusicForArtist]

In [ ]:
def searchAll(mdb, minI=-1):
    nArtists = len(mdb.getArtists())
    for i,artistName in enumerate(mdb.getArtists()):
        if i <= minI:
            continue

        print('\n\n\n')
        print('-'*130)
        print(i,'/',nArtists,'===>\t',artistName)
        print('-'*130)
        print('\n')
        
        dbdata = mdb.getArtistData(artistName)
        if all([dbdata.get("LastFM"), dbdata.get("Discogs"), dbdata.get("AllMusic"), dbdata.get("MusicBrainz")]):
            print("{0} is well known.".format(artistName))
            continue
        else:
            print("{0} get it!".format(artistName))

        for search in searches:
            print("====>",search)
            try:
                search(artistName)
            except:
                sleep(3)

In [ ]:
searchAll(mdb, 3404)

In [ ]:
# Refused

# Albums

In [ ]:
%load_ext autoreload
%autoreload
from album import album
from albums import albums
from discogsBase import discogs

disc = discogs()
albs = albums(disc)
#albs.parseAlbumModValFiles(modVal=27)

In [ ]:
def parseAlbums(modVal):
    albs.parseAlbumModValFiles(modVal=modVal)
    #albs.createAlbumModValMetadata(modVal)
    #albs.createAlbumModValMetadata(modVal=modVal)

def parseAlbumsParallel(modVals=None):
    if modVals is None:
        modVals = ["NAN"] + list(range(100))
        #modVals = range(100)
    nProcs = 2
    pool = Pool(processes=nProcs)
    result = pool.map_async(parseAlbums, modVals)
 
    start, cmt = clock("Processing {0} modVals using {1} processes".format(modVals, nProcs))
    print("Runing .",end="")
    while not result.ready():
        #print(".",end="")
        time.sleep(10)
    print("")
    
    elapsed(start, cmt)
    return result.get()

In [ ]:
%load_ext autoreload
%autoreload
from album import album
from albums import albums
albs = albums(disc)

start,cmt=clock("Parsing All Albums")
start2,cmt2=clock("Parallel Album Parsing")
parseAlbumsParallel()
elapsed(start2, cmt2)
elapsed(start, cmt)

# Download Stuff

In [ ]:
mdb = myMusicDBMap(debug=True)
maindb = mainDB(mdb=mdb, create=False, debug=True)
dbdata = maindb.dbdata

In [ ]:
ifile = "unmatchedArtists4.p"
noMatches = getFile(ifile)
print(len(noMatches))
dbs = ['Discogs', 'AllMusic', 'MusicBrainz', 'LastFM']
dbs = ['MusicBrainz']

#noMatches=['Go-A', 'PM', 'Qango']

from time import sleep
maxI = -1
for i,artist in enumerate(noMatches):
    print("="*100)
    print("="*100)
    print("="*100)
    print("="*100)
    print(i,'/',len(noMatches),'\t',artist)
    if i <= maxI:
        continue
    for db in dbs:
        print("="*100)
        print(i,'/',len(noMatches),'\t',artist,'===>',db)
        try:
            dbdata[db]["Artists"].searchForArtist(artist)
        except:
            sleep(5)